# 01｜重新出发，让我们学会和AI说话

## 1 获取 API Key

步骤略，自行找攻略。将生成的 API KEY 设置到环境变量中。

## 2 搭建开发环境

### 本地环境

使用 Python + JupyterNoteBook，IDE 使用 PyCharm，然后安装下面的依赖即可：

```shell
conda create --name py310 python=3.10
conda activate py310
conda install -c conda-forge jupyterlab
conda install -c conda-forge ipywidgets
conda install -c conda-forge openai
```

当然也可以使用 pip 来安装这些依赖。

### 通过 Colab 使用 JupyterLab

[Colab](https://colab.research.google.com/)，不想折腾本地环境的，还有一个选择，就是使用 Google 提供的叫做 Colab 的线上 Python Notebook 环境。

另外，即使你有本地环境，也建议创建一个 Colab 账号，因为 Colab 可以让你免费使用一些 GPU 的资源，在你需要使用 GPU 尝试训练一些深度学习模型，而又没有一张比较好的显卡的时候，就可以直接使用它。另一方面，Colab 便于你在网络上把自己撰写的 Python 代码分享给其他人。

Colab 已经是一个 Python Notebook 的环境了。只需要安装 openai 的库即可：

```shell
!pip install openai
%env OPENAI_API_KEY=在这里写你获取到的ApiKey
```

## 3 体验并测试 OpenAI 的 API

尝试运行下面代码

In [6]:
# 准备一下环境
from jproperties import Properties
import openai
import os

configs = Properties()
with open("../private.properties", 'rb') as config_file:
    configs.load(config_file)

# 设置 OpenAI 的 Key
openai.api_key = configs.get("open_key").data

# 解决 SSLError，参考 https://www.zhihu.com/question/588395420/answer/2928476066。
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

# 封装 open 的请求过程
COMPLETION_MODEL = "text-davinci-003"
def get_response(prompt):
    completions = openai.Completion.create(
        engine=COMPLETION_MODEL,
        prompt=prompt,
        max_tokens=512,
        n=1,
        stop=None,
        temperature=0.0,
    )
    message = completions.choices[0].text
    return message

In [2]:
import openai


test_prompt = """
Consideration product : 工厂现货PVC充气青蛙夜市地摊热卖充气玩具发光蛙儿童水上玩具

1. Compose human readable product title used on Amazon in english within 20 words.
2. Write 5 selling points for the products in Amazon.
3. Evaluate a price range for this product in U.S.

Output the result in json format with three properties called title, selling_points and price_range
"""


print(get_response(test_prompt))


{
    "title": "PVC Inflatable Glow-in-the-Dark Frog Water Toy for Kids Night Market Stall Hot Sale",
    "selling_points": [
        "Made of durable PVC material",
        "Inflatable design for easy storage and transport",
        "Glow-in-the-dark feature for night time fun",
        "Perfect for water play and pool parties",
        "Great for kids of all ages"
    ],
    "price_range": "$10 - $20"
}


上面商品并非虚构，而是直接找了 1688 里一个真实存在的商品。

这个实例中，我们调用了 OpenAI 的 Completion 接口，然后向它提了一个需求，也就是为一个 1688 上找到的中文商品名称做三件事情。

1. 为这个商品写一个适合在亚马逊上使用的英文标题。
2. 给这个商品写 5 个卖点。
3. 估计一下，这个商品在美国卖多少钱比较合适。

同时，我们告诉 OpenAI，我们希望返回的结果是 JSON 格式的，并且上面的三个事情用 title、selling_points 和 price_range 三个字段返回。从上面输出可以看出，OpenAI 真的理解了我们的需求，返回了一个符合我们要求的 JSON 字符串给我们。在这个过程中，它完成了好几件不同的事情。

1. 第一个是翻译，我们给的商品名称是中文的，返回的内容是英文的；
2. 第二个是理解你的语义去生成文本；
3. 第三个是利用 AI 自己有的知识给商品定价。【而如果用 “Glow-in-the-Dark frog” 在亚马逊里搜索，搜索结果的第一行里，就有一个 16 美元发光的青蛙，可以 AI 给出的价格是合理的】
4. 最后是根据我们的要求把我们想要的结果，通过一个 JSON 结构化地返回给我们。

## 4 第二个例子

In [7]:
test_prompt = """Man Utd must win trophies, says Ten Hag ahead of League Cup final请将上面这句话的人名提取出来，并用json的方式展示出来"""

print(get_response(test_prompt))



{
    "Ten Hag": "Ten Hag"
}


上面给了它一个英文的体育新闻的标题，然后让 AI 把其中的人名提取出来。可以看到，返回的结果也准确地把新闻里面唯一出现的人名——曼联队的主教练滕哈格的名字提取了出来。这个例子里，我们希望 AI 处理的内容是英文，给出的指令则是中文。不过 AI 都处理得很好，而且我们的输入完全是自然的中英文混合在一起，并没有使用特定的标识符或者分隔符。

## 5 内热总结

这里的两个例子，其实对应着很多不同的问题，其中就包括**机器翻译**、**文本生成**、**知识推理**、**命名实体识别**等等。在传统的机器学习领域，对于其中任何一个问题，都可能需要一个独立的机器学习模型。就算把这些模型都免费提供给你，把这些独立的机器学习模型组合到一起实现上面的效果，还需要海量的工程研发工作。没有一个数十人的团队，工作量根本看不到头。然而，OpenAI 通过一个包含 1750 亿参数的大语言模型，就能理解自然的语言输入，直接完成各种不同的问题。而这个让人惊艳的表现，也是让很多人惊呼“通用人工智能（AGI）要来了”的原因。

## 6 推荐阅读

- [GPT，GPT-2，GPT-3 论文精读【论文精读】](https://www.bilibili.com/video/BV1AF411b7xQ/)
